# MHE



In [1]:
!pip install do-mpc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!apt install imagemagick

Reading package lists... Done
Building dependency tree       
Reading state information... Done
imagemagick is already the newest version (8:6.9.7.4+dfsg-16ubuntu6.13).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [3]:
import numpy as np
from casadi import *

# Add do_mpc to path. This is not necessary if it was installed via pip.
import sys
sys.path.append('../../')

# Import do_mpc package:
import do_mpc

## Creating the model

In the following we will present the configuration, setup and connection between these blocks, starting with the `model`.
The considered model of the CSTR is continuous and has 4 states and 2 control inputs.
The model is initiated by:

In [4]:
model_type = 'continuous' # either 'discrete' or 'continuous'
model = do_mpc.model.Model(model_type)

The model is based on the assumption that we have additive process and/or measurement noise:

\begin{align}
\dot{x}(t) &= f(x(t),u(t),z(t),p(t),p_{\text{tv}}(t))+w(t), \\
y(t) &= h(x(t),u(t),z(t),p(t),p_{\text{tv}}(t))+v(t),
\end{align}

we are free to chose, which states and which measurements experience additive noise.

### Model variables

The next step is to define the model variables. It is important to define the variable type, name and optionally shape (default is scalar variable). 


### States and control inputs

The four states are concentration of reactant A ($C_{\text{A}}$), the concentration of reactant B ($C_{\text{B}}$), the temperature inside the reactor ($T_{\text{R}}$) and the temperature of the cooling jacket ($T_{\text{K}}$):

In [5]:
# States struct (optimization variables):
C_a = model.set_variable(var_type='_x', var_name='C_a', shape=(1,1))
C_b = model.set_variable(var_type='_x', var_name='C_b', shape=(1,1))
T_R = model.set_variable(var_type='_x', var_name='T_R', shape=(1,1))
T_K = model.set_variable(var_type='_x', var_name='T_K', shape=(1,1))

The control inputs are the feed $F$ and the heat flow $\dot{Q}$:

In [6]:
# Input struct (optimization variables):
F = model.set_variable(var_type='_u', var_name='F')
Q_dot = model.set_variable(var_type='_u', var_name='Q_dot')

### Model measurements

This step is essential for the state estimation task: We must define a measurable output.
Typically, this is a subset of states (or a transformation thereof) as well as the inputs.

Note that some MHE implementations consider inputs separately.

As mentionned above, we need to define for each measurement if additive noise is present.
In our case we can assume noisy state measurements ($\phi$) but perfect input measurements.

In [7]:
# State measurements
C_b_meas = model.set_meas('C_b_1_meas', C_b, meas_noise=True)
T_R_meas = model.set_meas('T_R_1_meas', T_R, meas_noise=True)
T_K_meas = model.set_meas('T_K_1_meas', T_K, meas_noise=True)

# Input measurements
F_meas = model.set_meas('F_meas', F, meas_noise=False)
Q_dot_meas = model.set_meas('Q_dot', Q_dot, meas_noise=False)

### Model parameters

Next we **define parameters**. The MHE allows to estimate parameters as well as states. Note that not all parameters must be estimated (as shown in the MHE setup below). We can also hardcode parameters.



### ODE and parameters

The system model is described by the ordinary differential equation:

\begin{align}
\dot{C}_{\text{A}} &= F \cdot \alpha \cdot (C_{\text{A},0} - C_{\text{A}}) - k_1 \cdot C_{\text{A}} - k_3 \cdot C_{\text{A}}^2, \\
\dot{C}_{\text{B}} &= F \cdot C_{\text{B}} + k_1 \cdot C_{\text{A}} - k_2 \cdot C_{\text{B}}, \\
\dot{T}_{\text{R}} &= \frac{k_1 \cdot C_{\text{A}} \cdot H_{\text{R},ab} + k_2 \cdot C_{\text{B}} \cdot  H_{\text{R},bc} + k_3 \cdot C_{\text{A}}^2 \cdot H_{\text{R},ad}} {-\rho \cdot \beta \cdot c_p}\\
&+ F \cdot (T_{\text{in}} - T_{\text{R}}) + \frac{K_w \cdot A_{\text{R}} \cdot(T_{\text{K}}-T_{\text{R}})}{\rho \cdot \beta \cdot c_p \cdot V_{\text{R}}}, \\
\dot{T}_{\text{K}} &= \frac{\dot{Q} + k_w \cdot A_{\text{R}} \cdot T_{\text{dif}}}{m_k \cdot C_{p,k}},
\end{align}

where

\begin{align}
k_1 &= \gamma \cdot k_{0,\text{ab}} \cdot \exp\left(\frac{-E_{\text{A},\text{ab}}}{T_{\text{R}}+273.15}\right), \\
k_1 &= k_{0,\text{bc}} \cdot \exp \left( \frac{-E_{\text{A},\text{bc}}}{T_{\text{R}}+273.15} \right), \\
k_3 &= k_{0,\text{ad}} \cdot \exp \left( \frac{-E_{\text{A},\text{ad}}}{T_{\text{R}}+273.15} \right).
\end{align}

The 
parameters $\alpha$ and $\beta$ and $\gamma$ are uncertain while the rest of the parameters is considered certain:

In [8]:
# Certain parameters
K0_ab = 1.287e12 # K0 [h^-1]
K0_bc = 1.287e12 # K0 [h^-1]
K0_ad = 9.043e9 # K0 [l/mol.h]
R_gas = 8.3144621e-3 # Universal gas constant
E_A_ab = 9758.3*1.00 #* R_gas# [kj/mol]
E_A_bc = 9758.3*1.00 #* R_gas# [kj/mol]
E_A_ad = 8560.0*1.0 #* R_gas# [kj/mol]
H_R_ab = 4.2 # [kj/mol A]
H_R_bc = -11.0 # [kj/mol B] Exothermic
H_R_ad = -41.85 # [kj/mol A] Exothermic
Rou = 0.9342 # Density [kg/l]
Cp = 3.01 # Specific Heat capacity [kj/Kg.K]
Cp_k = 2.0 # Coolant heat capacity [kj/kg.k]
A_R = 0.215 # Area of reactor wall [m^2]
V_R = 10.01 #0.01 # Volume of reactor [l]
m_k = 5.0 # Coolant mass[kg]
T_in = 130.0 # Temp of inflow [Celsius]
K_w = 4032.0 # [kj/h.m^2.K]
C_A0 = (5.7+4.5)/2.0*1.0 # Concentration of A in input Upper bound 5.7 lower bound 4.5 [mol/l]

# Uncertain parameters:
alpha = model.set_variable(var_type='parameter', var_name='alpha')
beta = model.set_variable(var_type='parameter', var_name='beta')
gamma = model.set_variable(var_type='parameter', var_name='gamma')

In the next step, we formulate the $k_i$-s:

In [9]:
# Auxiliary terms
K_1 = gamma * K0_ab * exp((-E_A_ab)/((T_R+273.15)))
K_2 =  K0_bc * exp((-E_A_bc)/((T_R+273.15)))
K_3 = K0_ad * exp((-E_A_ad)/((T_R+273.15)))

Additionally, we define an artificial variable of interest, that is not a state of the system, but will be later used for plotting:

In [10]:
T_dif = model.set_expression(expr_name='T_dif', expr=T_R-T_K)

### Right-hand-side equation
Finally, we set the right-hand-side of the model by calling `model.set_rhs(var_name, expr)` with the `var_name` from the state variables defined above.

Note that we can decide whether the inidividual states experience process noise. 
In this example we choose that the system model is perfect.
This is the default setting, so we don't need to pass this parameter explictly.

WIth the help ot the $k_i$-s and $T_{\text{dif}}$ we can define the ODEs:

In [11]:
model.set_rhs('C_a', F*alpha*(C_A0 - C_a) -K_1*C_a - K_3*(C_a**2), process_noise = False)
model.set_rhs('C_b', -F*C_b + K_1*C_a - K_2*C_b)
model.set_rhs('T_R', ((K_1*C_a*H_R_ab + K_2*C_b*H_R_bc + K_3*(C_a**2)*H_R_ad)/(-Rou*beta*Cp)) + F*(T_in-T_R) +(((K_w*A_R)*(-T_dif))/(Rou*beta*Cp*V_R)))
model.set_rhs('T_K', (Q_dot + K_w*A_R*(T_dif))/(m_k*Cp_k))

The model setup is completed by calling `model.setup()`:

In [12]:
model.setup()

After calling `model.setup()` we cannot define further variables etc.

## Configuring the moving horizon estimator
The first step of configuring the moving horizon estimator is to call the class with a list of all parameters to be estimated. An empty list (default value) means that no parameters are estimated.
The list of estimated parameters must be a subset (or all) of the previously defined parameters.

<div class="alert alert-info">
**Note** 
    
WE CAN DEFINE MORE UNCERTAIN PARAMETERS (BEYOND $\alpha , \beta , \gamma$) THAT WE DON'T ESTIMATE

In many cases we will use the same model for (robust) control and MHE estimation. In that case it is possible to have some external parameters (e.g. weather prediction) that are uncertain but cannot be estimated. 
</div>

In [13]:
mhe = do_mpc.estimator.MHE(model, ['alpha', 'beta', 'gamma'])

### MHE parameters:
Next, we pass MHE parameters. Most importantly, we need to set the time step and the horizon.
We also choose to obtain the measurement from the MHE data object. 
Alternatively, we are able to set a user defined measurement function that is called at each timestep and returns the `N` previous measurements for the estimation step.

In [14]:
setup_mhe = {
    't_step': 0.1,
    'n_horizon': 10,
    'store_full_solution': True,
    'meas_from_data': True
}
mhe.set_param(**setup_mhe)

### Objective function
The most important step of the configuration is to define the objective function for the MHE problem:

\begin{align}
\underset{
    \begin{array}{c}
    \mathbf{x}_{0:N+1}, \mathbf{u}_{0:N}, p,\\
    \mathbf{w}_{0:N}, \mathbf{v}_{0:N}
    \end{array}
    }{\mathrm{min}}
    &\frac{1}{2}\|x_0-\tilde{x}_0\|_{P_x}^2+\frac{1}{2}\|p-\tilde{p}\|_{P_p}^2
    +\sum_{k=0}^{N-1} \left(\frac{1}{2}\|v_k\|_{P_{v,k}}^2
    + \frac{1}{2}\|w_k\|_{P_{w,k}}^2\right),\\
    &\left.\begin{aligned}
    \mathrm{s.t.}\quad
    x_{k+1} &= f(x_k,u_k,z_k,p,p_{\text{tv},k})+ w_k,\\
    y_k &= h(x_k,u_k,z_k,p,p_{\text{tv},k}) + v_k, \\
    &g(x_k,u_k,z_k,p_k,p_{\text{tv},k}) \leq 0
    \end{aligned}\right\} k=0,\dots, N
\end{align}

We typically consider the formulation shown above, where the user has to pass the weighting matrices ``P_x``, ``P_v``, ``P_p`` and ``P_w``.
In our concrete example, we assume a perfect model without process noise and thus ``P_w`` is not required. 

We set the objective function with the weighting matrices shown below:

In [15]:
np.diag(np.array([ ]))

array([], shape=(0, 0), dtype=float64)

In [61]:
np.eye(3)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [17]:
# P_v = np.diag(np.array([ ])) 
# #In the case that the do_mpc.model.Model is configured without measurement-noise 
# #(see do_mpc.model.Model.set_meas()) the parameter P_v is not required.
# P_x = np.eye(4) # 4 states
# P_p = 6*np.eye(3) #6 inputs: 4 states + 2 control

# mhe.set_default_objective(P_x, P_v, P_p)

In [16]:
P_v = np.diag(np.array([1, 1, 1])) #3 states measured with (meas_noise=True)
P_x = np.eye(4) # 4 states
P_p = 6*np.eye(3) #6 inputs: 4 states + 2 control

mhe.set_default_objective(P_x, P_v, P_p)

### Fixed parameters
If the model contains parameters and if we estimate only a subset of these parameters, it is required to pass a function that returns the value of the remaining parameters at each time step. 

Furthermore, this function must return a specific structure, which is first obtained by calling:

In [78]:
# p_template_mhe = mhe.get_p_template()

Using this structure, we then formulate the following function for the remaining (not estimated) parameters:

In [19]:
# def p_fun_mhe(t_now):
#     p_template_mhe['Theta_2'] = 2.25e-4
#     p_template_mhe['Theta_3'] = 2.25e-4
#     return p_template_mhe

This function is finally passed to the ``mhe`` instance:

In [20]:
# mhe.set_p_fun(p_fun_mhe)

### Bounds
The MHE implementation also supports bounds for states, inputs, parameters which can be set as shown below.
For the given example, it is especially important to set realistic bounds on the estimated parameter. Otherwise the MHE solution is a poor fit.

In [17]:
# lower bounds of the states
mhe.bounds['lower', '_x', 'C_a'] = 0.1
mhe.bounds['lower', '_x', 'C_b'] = 0.1
mhe.bounds['lower', '_x', 'T_R'] = 50
mhe.bounds['lower', '_x', 'T_K'] = 50

# upper bounds of the states
mhe.bounds['upper', '_x', 'C_a'] = 2
mhe.bounds['upper', '_x', 'C_b'] = 2
mhe.bounds['upper', '_x', 'T_K'] = 140

# lower bounds of the inputs
mhe.bounds['lower', '_u', 'F'] = 5
mhe.bounds['lower', '_u', 'Q_dot'] = -8500

# upper bounds of the inputs
mhe.bounds['upper', '_u', 'F'] = 100
mhe.bounds['upper', '_u', 'Q_dot'] = 0.0

### Setup
Similar to the controller, simulator and model, we finalize the MHE configuration by calling:

In [18]:
mhe.setup()

## Configuring the Simulator
In many cases, a developed control approach is first tested on a simulated system. **do-mpc** responds to this need with the `do_mpc.simulator` class. The `simulator` uses state-of-the-art DAE solvers, e.g. Sundials [CVODE](https://computing.llnl.gov/projects/sundials/cvode) to solve the DAE equations defined in the supplied `do_mpc.model`. This will often be the same model as defined for the `optimizer` but it is also possible to use a more complex model of the same system.

In this section we demonstrate how to setup the `simulator` class for the given example. We initialize the class with the previously defined `model`:

In [19]:
simulator = do_mpc.simulator.Simulator(model)

### Simulator parameters

Next, we need to parametrize the `simulator`. Please see the API documentation for `simulator.set_param()` for a full description of available parameters and their meaning. Many parameters already have suggested default values. Most importantly, we need to set `t_step`. We choose the same value as for the `optimizer`.

In [20]:
# Instead of supplying a dict with the splat operator (**), as with the optimizer.set_param(),
# we can also use keywords (and call the method multiple times, if necessary):
simulator.set_param(t_step = 0.1)

### Parameters
In the `model` we have defined the inertia of the masses as parameters. The `simulator` is now parametrized to simulate using the "true" values at each timestep. In the most general case, these values can change, which is why we need to supply a function that can be evaluted at each time to obtain the current values. 
**do-mpc** requires this function to have a specific return structure which we obtain first by calling:

In [21]:
p_template_sim = simulator.get_p_template()

We need to define a function which returns this structure with the desired numerical values. For our simple case:

In [22]:
def p_fun_sim(t_now):
    p_template_sim['alpha'] = 1
    p_template_sim['beta'] = 1
    p_template_sim['gamma'] = 1
    return p_template_sim

This function is now supplied to the `simulator` in the following way:

In [23]:
simulator.set_p_fun(p_fun_sim)

### Setup
Finally, we call:

In [24]:
simulator.setup()

## Creating the loop
While the full loop should also include a controller, we are currently only interested in showcasing the estimator. We therefore estimate the states for an arbitrary initial condition and some random control inputs (shown below).

In [25]:
x0 = np.pi*np.array([1.0, 0.1, 35, 30]).reshape(-1,1)

To make things more interesting we pass the estimator a perturbed initial state:

In [26]:
x0_mhe = x0*(1+0.5*np.random.randn(4,1))

and use the ``x0`` property of the simulator and estimator to set the initial state:

In [27]:
simulator.x0 = x0
mhe.x0_mhe = x0_mhe
mhe.p_est0['alpha'] = 5e-1
mhe.p_est0['beta'] = 5e-1
mhe.p_est0['gamma'] = 5e-1

# p_est0
# Class attribute.

# MHE.p_est0
# Initial value of estimated parameters and current iterate. 
# This is the numerical structure holding the information about the current estimated parameters in the class. 
# The property can be indexed according to the model definition.

It is also adviced to create an initial guess for the MHE optimization problem. The simplest way is to base that guess on the initial state, which is done automatically when calling:

In [28]:
mhe.set_initial_guess()

### Setting up the Graphic
We are again using the **do-mpc** `graphics` module. This versatile tool allows us to conveniently configure a user-defined plot based on Matplotlib and visualize the results stored in the `mhe.data`, `simulator.data` objects. 

We start by importing matplotlib:

In [29]:
import matplotlib.pyplot as plt
import matplotlib as mpl
# Customizing Matplotlib:
mpl.rcParams['font.size'] = 18
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['axes.grid'] = True

And initializing the `graphics module` with the data object of interest. 
In this particular example, we want to visualize both the `mpc.data` as well as the `simulator.data`.

In [30]:
mhe_graphics = do_mpc.graphics.Graphics(mhe.data)
sim_graphics = do_mpc.graphics.Graphics(simulator.data)

Next, we create a `figure` and obtain its `axis` object. Matplotlib offers multiple alternative ways to obtain an `axis` object, e.g. [subplots](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.subplots.html), [subplot2grid](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.subplot2grid.html), or simply [gca](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.gca.html). We use `subplots`:

In [31]:
%%capture 
# We just want to create the plot and not show it right now. This "inline magic" suppresses the output.
fig, ax = plt.subplots(5, sharex=True, figsize=(16,12))
fig.align_ylabels()

# We create another figure to plot the parameters:
fig_p, ax_p = plt.subplots(1, figsize=(16,4))

Most important API element for setting up the `graphics` module is `graphics.add_line`, which mimics the API of `model.add_variable`, except that we also need to pass an `axis`. 

We want to show both the simulator and MHE results on the same axis, which is why we configure both of them identically:

In [32]:
%%capture
for g in [sim_graphics, mhe_graphics]:
    # Plot the concentrations (Ca, Cb) on the first axis:
    g.add_line(var_type='_x', var_name='C_a', axis=ax[0])
    g.add_line(var_type='_x', var_name='C_b', axis=ax[0])
    ax[0].set_prop_cycle(None)

    # Plot the temperatures (T_R, T_K) on the second axis:    
    g.add_line(var_type='_x', var_name='T_R', axis=ax[1])
    g.add_line(var_type='_x', var_name='T_K', axis=ax[1])
    ax[1].set_prop_cycle(None)

    # Plot the T_dif on the third axis:
    g.add_line(var_type='_aux', var_name='T_dif', axis=ax[2])
    ax[2].set_prop_cycle(None)

    # Plot the Q_dot on the third axis:
    g.add_line(var_type='_u', var_name='Q_dot', axis=ax[3])
    ax[3].set_prop_cycle(None)  

    # Plot the F on the third axis:
    g.add_line(var_type='_u', var_name='F', axis=ax[4])
    ax[4].set_prop_cycle(None)      
    
    g.add_line(var_type='_p', var_name='alpha', axis=ax_p)
    g.add_line(var_type='_p', var_name='beta', axis=ax_p)
    g.add_line(var_type='_p', var_name='gamma', axis=ax_p)

ax[0].set_ylabel('C [mol/l]')
ax[1].set_ylabel('T [K]')
ax[2].set_ylabel('$\Delta$ T [K]')
ax[3].set_ylabel('Q [kW]')
ax[4].set_ylabel('Flow [L/h]')
ax[4].set_xlabel('time [h]')

Before we show any results we configure we further configure the graphic, by changing the appearance of the simulated lines. We can obtain line objects from any graphics instance with the ``result_lines`` property:

In [33]:
sim_graphics.result_lines

We obtain a structure that can be queried conveniently as follows: 

In [34]:
# First element for state Cb:
sim_graphics.result_lines['_x', 'C_b', 0]

In this particular case we want to change all ``result_lines`` with:

In [35]:
for line_i in sim_graphics.result_lines.full:
    line_i.set_alpha(0.4)
    line_i.set_linewidth(6)

We furthermore use this property to create a legend:

In [36]:
ax[0].legend(sim_graphics.result_lines['_x', 'C_b'], '123', title='Sim.', loc='center right')
ax[1].legend(mhe_graphics.result_lines['_x', 'C_b'], '123', title='MHE', loc='center right')

and another legend for the parameter plot:

In [37]:
ax_p.legend(sim_graphics.result_lines['_p', 'alpha', 'beta', 'gamma']+mhe_graphics.result_lines['_p', 'alpha', 'beta', 'gamma'], ['True','Estim.'])

### Running the loop
We investigate the closed-loop MHE performance by alternating a simulation step (`y0=simulator.make_step(u0)`) and an estimation step (`x0=mhe.make_step(y0)`). Since we are lacking the controller which would close the loop (`u0=mpc.make_step(x0)`), we define a random control input function:

In [38]:
def random_u(u0):
    # Hold the current value with 80% chance or switch to new random value.
    u_next = (0.5-np.random.rand(2,1))*np.pi # New candidate value.
    switch = np.random.rand() >= 0.8 # switching? 0 or 1.
    u0 = (1-switch)*u0 + switch*u_next # Old or new value.
    return u0

The function holds the current input value with 80% chance or switches to a new random input value.

We can now run the loop. At each iteration, **we perturb our measurements**,
for a more realistic scenario.
This can be done by calling the simulator with a value for the measurement noise, which we defined in the model above.

In [39]:
u0 = np.zeros((2,1))

In [40]:
u0

array([[0.],
       [0.]])

In [41]:
np.random.seed(999) #make it repeatable
v0 = 0.1*np.random.randn(model.n_v,1) # measurement noise

In [42]:
v0

array([[0.01271578],
       [0.14018909],
       [0.0314815 ]])

In [43]:
y0 = simulator.make_step(u0, v0=v0)

In [44]:
y0

array([[  1.05535729],
       [113.50480786],
       [112.86820108],
       [  0.        ],
       [  0.        ]])

In [ ]:
x0 = mhe.make_step(y0)

In [ ]:
%%capture
np.random.seed(999) #make it repeatable

u0 = np.zeros((2,1))
for i in range(50):
    u0 = random_u(u0) # Control input
    v0 = 0.1*np.random.randn(model.n_v,1) # measurement noise
    y0 = simulator.make_step(u0, v0=v0)
    x0 = mhe.make_step(y0) # MHE estimation step

We can visualize the resulting trajectory with the pre-defined graphic:

In [54]:
sim_graphics.plot_results()
mhe_graphics.plot_results()
# Reset the limits on all axes in graphic to show the data.
mhe_graphics.reset_axes()

# Mark the time after a full horizon is available to the MHE.
ax[0].axvline(1)
ax[1].axvline(1)
ax[2].axvline(1)
ax[3].axvline(1)
ax[4].axvline(1)
# Show the figure:
fig

AssertionError: ignored

Parameter estimation:

In [ ]:
ax_p.set_ylim(1e-4, 4e-4)
ax_p.set_ylabel('alpha, beta, gamma')
ax_p.set_xlabel('time [h]')
fig_p